## Objective: Main aim of this notebook is to generate face behind the mask using autoencoder.

## Import Necessary Libraries

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Input
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
import numpy as np
import os
import re
from skimage.measure import compare_ssim
from scipy import ndimage
import random


In [ ]:
print(tf.__version__)
print(keras.__version__)
print(np.__version__)

In [ ]:
from keras.engine import  Model
from keras.layers import Input

In [ ]:
!pip install image-quality
import imquality.brisque as brisque

## Load data
Here I am going to load only 2500 images each with mask and no mask. These images are converted to an array and are appended in empty array. Here I also have defind function to load data serially. 

In [ ]:
# to get the files in proper order
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)


# defining the size of image 
SIZE = 256

mask_path = '../input/face-mask-lite-dataset/with_mask'
mask_array = []

image_path = '../input/face-mask-lite-dataset/without_mask'
img_array = []

image_file = sorted_alphanumeric(os.listdir(image_path))
mask_file = sorted_alphanumeric(os.listdir(mask_path))

        
for i in tqdm(image_file):
 
    if i == 'seed1000.png':
        break
    
    else:
        image = cv2.imread(image_path + '/' + i,1)

        # as opencv load image in bgr format converting it to rgb
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # resizing images 
        image = cv2.resize(image, (SIZE, SIZE))

        # normalizing image 
        image = image.astype('float32') / 255.0
        # appending normal sketch image
        img_array.append(img_to_array(image))
    

In [ ]:
for i in tqdm(mask_file):
    if i == 'with-mask-default-mask-seed1000.png':
        break
    else:    
        image = cv2.imread(mask_path + '/' + i,1)

          # as opencv load image in bgr format converting it to rgb
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # resizing images 
        image = cv2.resize(image, (SIZE, SIZE))

        # normalizing image 
        image = image.astype('float32') / 255.0

        #appending normal normal image    
        mask_array.append(img_to_array(image))

## Plot image pair

## Slicing and reshaping
Here i have used 2300 images for training and remaining 200 for testing.

In [ ]:
n = int(len(mask_array)*0.8)
train_mask_image = mask_array[:n]
train_image = img_array[:n]
test_mask_image = mask_array[n:]
test_image = img_array[n:]

# reshaping
train_mask_image = np.reshape(train_mask_image,(len(train_mask_image),SIZE,SIZE,3))
train_image = np.reshape(train_image, (len(train_image),SIZE,SIZE,3))
print('Train no mask image shape:',train_image.shape)
test_mask_image = np.reshape(test_mask_image,(len(test_mask_image),SIZE,SIZE,3))
test_image = np.reshape(test_image, (len(test_image),SIZE,SIZE,3))
print('Test no mask image shape',test_image.shape)

In [ ]:
if 0:
    mask_region = [i==r for (i,r) in zip(mask_array, img_array)]

    mask_region = [tf.reduce_all(x,axis=2) for x in mask_region]
    mask_region = [tf.stack([tf.logical_not(x),x],axis=2) for x in mask_region]
    mask_region_float = tf.reshape([tf.reshape(m,(SIZE,SIZE,2)) for m in mask_region], (len(mask_region),SIZE,SIZE,2))


    print(img_array[0][:,:,1] - tf.cast(mask_region_float[0,:,:,1],dtype='float'))
    plt.imshow(img_array[0][:,:,0] -  tf.cast(mask_region_float[0,:,:,1],dtype='float'))
    plt.show()
    print(mask_region_float.shape)

In [ ]:
'''
train_mask_image.shape
print(mask_region_float.shape)
mask_region_tr = mask_region_float[:800,:,:,:]
mask_region_te = mask_region_float[800:,:,:,:]
'''

image editing

In [ ]:
def blurImage(im):
    k = random.randint(1,3)
    k = k * 2 + 1
    res = cv2.GaussianBlur(im,(k,k),0)
    return res

def medFilt(im):
    res = np.uint8(im*256)
    res = cv2.medianBlur(res,5)
    return res

def rotImage(im):   
    #rotation angle in degree
    (h, w) = im.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    deg = random.random() - 0.5
    isflip = random.random()
    M = cv2.getRotationMatrix2D((cX, cY), 20*deg, 1.0)
    res = cv2.warpAffine(im, M, (w, h))
    
    if isflip>0.5:
        res = flipImage(res)
    return res

def flipImage(im):
    res = np.fliplr(im)
    return res

def saltPImage(im):
    (h, w) = image.shape[:2]
    sc = random.random()
    m = np.random.normal(scale=sc/2,size = (h,w,3))
    res = im + m
    return res

def shotNoise(im):
    (h, w) = image.shape[:2]
    rng = np.random.default_rng()
    sc = random.random() / 10
    s = rng.poisson(sc, (h,w,3)) - sc
    res = im + s
    return res

def darkenIm(im):
    factor = random.random()
    res = im * (0.5+factor/2)
    return res

def brightenIm(im):
    factor = random.random()
    factor = factor / 2 + 0.5
    res = im / factor
    return res

def changeColour(im):
    order = np.random.permutation(range(3))
    res = im[:,:,[order[0], order[1], order[2]]]
    return res

def edit_images(mask_array):
    mask_array_edit = mask_array.copy()
    for iter in range(3):
        for ind in tqdm(range(len(mask_array))):
            im = mask_array_edit[ind]
            alg = random.random()
            if alg < 0.1:
                res = im
            elif alg < 0.2 and alg > 0.1:
                res = blurImage(im)
                res = im
            elif alg < 0.3 and alg > 0.2:
                # res = medFilt(im)
                res = im
            elif alg < 0.4 and alg > 0.3:
                res = rotImage(im)
            elif alg < 0.5 and alg > 0.4:
                res = saltPImage(im)
            elif alg < 0.6 and alg > 0.5:
                res = changeColour(im)
            elif alg < 0.7 and alg > 0.6:
                res = darkenIm(im)
            elif alg < 0.8 and alg > 0.7:
                res = shotNoise(im)
            elif alg < 0.9 and alg > 0.8:
                res = im
                res = brightenIm(im)
            else:
                res = flipImage(im)
            mask_array_edit[ind] = res
    n = int(np.round(len(mask_array_edit)*0.8))
    mask_array_edit_tr = mask_array_edit[:n]
    mask_array_edit_te = mask_array_edit[n:]

    mask_array_edit_tr = np.reshape(mask_array_edit_tr,(len(mask_array_edit_tr),SIZE,SIZE,3))
    print('Train no mask image shape:',mask_array_edit_tr.shape)

    mask_array_edit_te = np.reshape(mask_array_edit_te,(len(mask_array_edit_te),SIZE,SIZE,3))
    print('Test no mask image shape',mask_array_edit_te.shape)
    return mask_array_edit_tr,mask_array_edit_te, mask_array_edit

mask_array_edit_tr,mask_array_edit_te, mask_array_edit = edit_images(mask_array)

In [ ]:
def plot_image_pair(images = 5):
    for i in range(images):
        plt.figure(figsize = (7,7))
        plt.subplot(1,3,1)
        plt.title("No Mask", fontsize = 15)
        plt.imshow(img_array[i].reshape(SIZE, SIZE, 3))
        plt.subplot(1,3,2)
        plt.title("Mask", fontsize = 15)
        plt.imshow(mask_array[i].reshape(SIZE, SIZE, 3))
        plt.subplot(1,3,3)
        plt.title("Mask", fontsize = 15)
        plt.imshow(mask_array_edit[i].reshape(SIZE, SIZE, 3))


In [ ]:
plot_image_pair(20)

## Defining our model

In [ ]:
encoder_input = keras.Input(shape=(SIZE,SIZE, 3), name="img")
x = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(encoder_input)
x = MaxPool2D(pool_size = (2,2))(x)

x = Conv2D(filters = 32,kernel_size = (3,3),strides = (2,2), activation = 'relu', padding = 'valid')(x)
x = Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), activation = 'relu', padding = 'same')(x)
x = MaxPool2D(pool_size = (2,2))(x)

x = Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
x = Conv2D(filters = 256 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
encoder_output = Conv2D(filters = 512 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
encoder = tf.keras.Model(encoder_input, encoder_output)

decoder_input = Conv2D(filters = 512 ,kernel_size = (3,3), activation = 'relu', padding = 'same')(encoder_output)
x = UpSampling2D(size = (2,2))(decoder_input)
x = Conv2D(filters = 256, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 164, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)

x = Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)
x = Conv2D(filters = 32 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)
 
x = Conv2D(filters = 16  , kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
decoder_output = Conv2D(filters = 3, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)

# final model
model = keras.Model(encoder_input, decoder_output)
model.summary()

In [ ]:
"""
encoder_input = keras.Input(shape=(SIZE,SIZE, 3), name="img")
x = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(encoder_input)
x = MaxPool2D(pool_size = (2,2))(x)

x = Conv2D(filters = 32,kernel_size = (3,3),strides = (2,2), activation = 'relu', padding = 'valid')(x)
x = Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), activation = 'relu', padding = 'same')(x)
x = MaxPool2D(pool_size = (2,2))(x)

x = Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
x = Conv2D(filters = 256 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
encoder_output = Conv2D(filters = 512 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x) 
encoder = tf.keras.Model(encoder_input, encoder_output)

decoder_input = Conv2D(filters = 512 ,kernel_size = (3,3), activation = 'relu', padding = 'same')(encoder_output)
x = UpSampling2D(size = (2,2))(decoder_input)
x = Conv2D(filters = 256, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(filters = 164, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)

x = Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)
x = Conv2D(filters = 32 , kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D(size = (2,2) )(x)
 
x = Conv2D(filters = 16  , kernel_size = (3,3), activation = 'relu', padding = 'same')(x)
decoder_output = Conv2D(filters = 2, kernel_size = (3,3), activation = 'relu', padding = 'same')(x)

# final model
maskModel = keras.Model(encoder_input, decoder_output)
maskModel.summary()
"""

FOR GAN: defining our descriminator:

In [ ]:
'''
# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model
'''

## Compiling our model

In [ ]:
# create a loss
def myLoss(im,ref):
    mse = keras.losses.mean_squared_error(im,ref)
    score = tf.image.ssim_multiscale(im, ref,1,filter_sigma=0.75)
    return tf.reduce_mean(mse) * (tf.abs(1-score[0]))

In [ ]:
tf.shape(train_image)
#prediction = maskModel.predict(mask_array[0].reshape(1,SIZE, SIZE, 3))
#plt.imshow(np.squeeze(prediction))


In [ ]:
'''
maskModel.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), 
                  loss = keras.losses.categorical_crossentropy,
              metrics = 'acc')


maskModel.fit(train_image, mask_region_tr, epochs = 10)
'''

In [ ]:
    print(train_image[0].shape) 
    prediction = model.predict(tf.reshape(train_image[1],(-1,256,256,3)))
    plt.imshow(tf.squeeze(prediction))
    plt.show()

In [ ]:
print(train_image.dtype)
def plot_images(start = 0, end = 5):
    #  os.mkdir('/kaggle/working/comps')
    for i in range(start, end, 1):
        plt.figure(figsize = (10,10))
        plt.subplot(1,3,1)
        plt.title("No Mask", fontsize = 12)
        plt.imshow(test_image[i])
        plt.subplot(1,3,2)
        plt.title("Mask", fontsize = 12)
        plt.imshow(mask_array_edit_te[i])
        plt.subplot(1,3,3)
        plt.title("Predicted", fontsize = 12)
        prediction = model.predict(tf.reshape(tf.keras.backend.constant(mask_array_edit_te[i]),
                                              (-1,256,256,3)))
        plt.imshow(tf.squeeze(prediction))
        
        #  plt.savefig('comps'+os.sep+'test img:'+str(i)+'.tiff')
        plt.show()
        

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mse',
              metrics = ['acc'])
results=[]

for iter in range(5):
    mask_array_edit_tr,mask_array_edit_te,mask_array_edit = edit_images(mask_array)
    model.fit(mask_array_edit_tr, train_image, epochs = 15,shuffle='True',validation_split = 0.2)
    plot_images(21,25)
    results.append(model.evaluate(mask_array_edit_te, test_image, batch_size=32))
    print("test loss, test acc:", results[iter])
    print("----"+str(iter)+"-----")

## Model evaluation

In [ ]:
loss_acc= model.evaluate(test_mask_image, test_image)
print("Loss: ",loss_acc[0])
print('Accuracy: ', np.round(loss_acc[1],2) * 100)

**discriminator**

In [ ]:
from keras.applications.vgg16 import VGG16
vgg_disc = VGG16(include_top=0,input_shape=(256,256,3),classes=1000) # real, fake
print(vgg_disc.summary())


In [ ]:
vgg_out_shape = (8,8,512)

for layer in vgg_disc.layers[:]:
    layer.trainable = False
    
discriminator = keras.Sequential()

# Add the vgg convolutional base model
discriminator.add(vgg_disc)
# Add new layers
discriminator.add(keras.layers.Flatten())
discriminator.add(Dense(128, activation='relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(2, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
discriminator.summary()

discriminator.compile(loss='mae',
              optimizer=keras.optimizers.Adam(lr=1e-6),
              metrics=['acc'])

In [ ]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
im = tf.reshape(preprocess_input(train_image[0]),(-1,256,256,3))

yhat = discriminator.predict(im)
print(yhat)

training of discriminator

In [ ]:
print(len(mask_array_edit))

In [ ]:
gen_images = np.zeros((len(mask_array_edit),SIZE,SIZE,3))
for i in tqdm(range(len(mask_array_edit))):
    gen_images[i,:,:,:] = np.reshape(model.predict(tf.reshape(mask_array_edit[i],
                                                              (1,SIZE,SIZE,3))),(1,SIZE,SIZE,3))       

gen_images = tf.Variable(gen_images)

In [ ]:
train_ims_tf = tf.Variable(np.reshape(train_image,(-1,256,256,3)))

In [ ]:
for i in range(10):
    plt.imshow(test_image[i])
    plt.show()
    plt.imshow(gen_images[len(train_image) + i])
    plt.show()

In [ ]:
print(tf.shape(gen_images))
print(tf.shape(train_ims_tf)) # list of tensors

In [ ]:
disc_data = tf.concat([tf.cast(gen_images,dtype='float32'), tf.cast(train_ims_tf,dtype='float32')],0)
print(tf.shape(disc_data))
nims = tf.shape(gen_images)[0]
nims2 = tf.shape(train_ims_tf)[0]

disc_tags = tf.concat([tf.concat([tf.zeros((nims,1),dtype='int32'),tf.ones((nims,1),dtype='int32')],1), 
             tf.concat([tf.ones((nims2,1),dtype='int32'),tf.zeros((nims2,1),dtype='int32')],1)],0)
print(disc_tags[0])
print(disc_tags[1500])
print(tf.shape(disc_tags))

In [ ]:
discriminator.fit(disc_data, disc_tags,batch_size=64,epochs=10, validation_split = 0.2,shuffle='True')

Test the discriminator

In [ ]:
preds = []
for i in tqdm(range(tf.shape(gen_images)[0])):
    preds.append(discriminator(tf.reshape(gen_images[i],(-1,256,256,3))))



In [ ]:
preds = np.asarray(preds)
vals = np.argsort(-preds[:,0][:,0])
print(vals[0:4])

for iter in range(20):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(gen_images[vals[iter]])
    plt.title(preds[vals[iter]])
    plt.subplot(1,2,2)
    if vals[iter] < len(train_image):
        plt.imshow(train_image[vals[iter]])
        plt.show()

finally, train our model with discriminator as loss

In [ ]:
def discLoss(img,truth):
    res1 = discriminator(img)
    res2 = discriminator(truth)
    res3 = keras.losses.MSE(img,truth)
    return tf.norm(res1-res2)*0.2 + res3*0.8

In [ ]:
def brisqueLoss(img, truth):
    res1 = brisque.score(tf.make_ndarray(img))
    res2 = brisque.score(tf.make_ndarray(truth))
    res3 = keras.losses.MSE(img,truth)
    return abs(res1-res2)*0.2 + res3*0.8

In [ ]:
def ssimLoss(img,truth):
    res1 = tf.image.ssim_multiscale(img,truth,255)
    res3 = keras.losses.MSE(img,truth)
    return 1e3*(1-res1)*0.2 + res3*0.8

In [ ]:
def psnrLoss(img,truth):
    res1 = tf.image.psnr(img,truth,255)
    res2 = tf.image.ssim_multiscale(img,truth,255)
    return 10/res1*0.2 + 1e3*(1-res2)

In [ ]:
for layer in discriminator.layers[:]:
    layer.trainable = False

try to train a new generator

In [ ]:
from copy import copy
editor = copy(model)


In [ ]:
editor.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5), 
               loss = 'mse', metrics = ['acc']) 
gen_train = np.array(gen_images[0:500]) 
train_ims_tf2 = np.array(train_ims_tf[0:500])
editor.fit(gen_train, train_ims_tf2, epochs = 5,validation_split = 0.2,shuffle='True')

In [ ]:
for iter in range(20):

    if vals[iter] < len(train_image):
        plt.figure()
        plt.subplot(1,2,1)
        plt.imshow(tf.squeeze(editor(tf.reshape(gen_images[vals[iter]],(-1,256,256,3))) ))
        plt.title(preds[vals[iter]])
        plt.subplot(1,2,2)
        plt.imshow(train_image[vals[iter]])
        plt.show()
    print(editor(tf.reshape(gen_images[vals[iter]],(-1,256,256,3))))

In [ ]:
if True:
    for iter in range(10):
        mask_array_edit_tr,mask_array_edit_te,mask_array_edit = edit_images(mask_array)
        
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-6), loss = psnrLoss,
                  metrics = ['acc'])
        model.fit(mask_array_edit_tr, train_image, epochs = 5,shuffle='True',validation_split = 0.2)
        
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-6), loss = discLoss,
                     metrics = ['acc'])
        model.fit(mask_array_edit_tr, train_image, epochs = 5,shuffle='True',validation_split = 0.2, 
                  batch_size=16)
        
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5), loss = 'mse',
                  metrics = ['acc'])
        model.fit(mask_array_edit_tr, train_image, epochs = 5,validation_split = 0.2,shuffle='True')
        plot_images(50,55)
    plot_images(20,30)

## plotting images

In [ ]:
def plot_images(start = 0, end = 5):
    for i in range(start, end, 1):
        plt.figure(figsize = (10,10))
        plt.subplot(1,3,1)
        plt.title("No Mask", fontsize = 12)
        plt.imshow(test_image[i])
        plt.subplot(1,3,2)
        plt.title("Mask", fontsize = 12)
        plt.imshow(test_mask_image[i])
        plt.subplot(1,3,3)
        plt.title("Predicted", fontsize = 12)
        prediction = model.predict(test_mask_image[i].reshape(1,SIZE, SIZE, 3)).reshape(SIZE, SIZE, 3)
        plt.imshow(prediction)
        plt.show()
        

In [ ]:
plot_images(5,12)

In [ ]:

'''
mask_path = '../input/toker2/'
mask_array = []


mask_file = sorted_alphanumeric(os.listdir(mask_path))
for i in tqdm(mask_file):
    image = cv2.imread(mask_path + '/' + i,1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (SIZE, SIZE))
    image = image.astype('float32') / 255.0
    mask_array.append(img_to_array(image))
    
'''


In [ ]:
def plot_image_pair(images = 1):
    for i in range(images):
        plt.figure(figsize = (7,7))
        plt.subplot(1,2,1)
        plt.title("Mask", fontsize = 15)
        plt.imshow(mask_array[i].reshape(SIZE, SIZE, 3))
        plt.subplot(1,2,2)
        plt.title("NO Mask", fontsize = 15)
        plt.imshow(model.predict(mask_array[i].reshape(1,SIZE, SIZE, 3)).reshape(SIZE, SIZE, 3))

        
        
        
plot_image_pair(7)

In [ ]:
if 0:
    import os
    os.chdir(r'/kaggle/working/comps')
    print(os.listdir())
    import shutil
    shutil.make_archive('comparison', 'zip', os.getcwd())

In [ ]:
if  0: 
    from IPython.display import FileLink
    FileLink(r'./comparison.zip')

In [ ]:
prediction = model.predict(mask_array[0].reshape(1,SIZE, SIZE, 3)).reshape(SIZE, SIZE, 3)
plt.imshow(prediction)

# Thanks for your visit